In [1]:
import pandas as pd

In [2]:
def convert_to_int(x_str):
    # print(x_str)
    a = x_str.split(".")
    a.extend(a.pop(-1).split(","))
    if len(a) == 1:
        return int(a[0])
    if len(a) == 2:
        return int(a[0])+float(f"0.{a[-1]}")
    else:
        return int(a[0])*1000+int(a[1])+float(f"0.{a[-1]}")

In [3]:
df_net = pd.read_csv(
    "../data/Netzeinspeisung_2019.csv", 
    sep=";",
    skiprows=4
).dropna(axis=1)
df_net["MW"] = df_net["MW"].apply(lambda x: x*1000)

In [4]:
df_sol = pd.read_csv(
    "../data/Solarenergie_Prognose_2019.csv", 
    sep=";",
    skiprows=4
).dropna(axis=1)
df_sol["MW"] = df_sol["MW"].apply(convert_to_int)

In [5]:
df_win = pd.read_csv(
    "../data/Windenergie_Prognose_2019.csv", 
    sep=";",
    skiprows=4
).dropna(axis=1)
df_win = df_win.drop(["Onshore MW", "Offshore MW"], axis=1)
df_win["MW"] = df_win["MW"].apply(convert_to_int)

In [6]:
df_comb = df_net
df_comb["MW_sol"] = df_sol["MW"]
df_comb["MW_win"] = df_win["MW"]

In [7]:
df_comb["%"] = (df_comb["MW_win"] + df_comb["MW_sol"]) / df_comb["MW"]

## problem: 101%
For some points in time the % seems to be more than 100%, which should not be possible. It happens in 3.125% of the cases. <br>
We couldn't figure out the problem, so we just ignore for the moment...

In [8]:
df_comb.where(df_comb["%"] >= 1.0).dropna()

,Datum,Von,bis,MW,MW_sol,MW_win,%
37,01.01.2019,09:15,09:30,13968.0,118.464,13971.368,1.008722
38,01.01.2019,09:30,09:45,13937.0,166.057,14020.830,1.017930
39,01.01.2019,09:45,10:00,13944.0,216.277,14054.078,1.023405
40,01.01.2019,10:00,10:15,14110.0,270.314,14084.599,1.017357
41,01.01.2019,10:15,10:30,14056.0,319.318,14055.328,1.022670
...,...,...,...,...,...,...,...
33462,15.12.2019,13:30,13:45,13150.0,1068.541,12888.052,1.061338
33463,15.12.2019,13:45,14:00,13055.0,968.139,12709.125,1.047665
33464,15.12.2019,14:00,14:15,13077.0,855.921,12521.510,1.022974
33465,15.12.2019,14:15,14:30,12877.0,757.802,12413.655,1.022867


In [9]:
import plotly.graph_objects as go